In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/3.part'):
    shutil.rmtree('./file/3.part')

os.makedirs('./file/3.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.url/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        request_url = input_.loc[a, 'Url']
                        
                        resp = requests.get(request_url,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))

                            if html.xpath('//h3/text()')[0].strip() == 'Can\'t find the item you are looking for on this list? Download this':
                                break
                
                            title = html.xpath('//h1[@itemprop="name"]/text()')[0].strip()
                            if html.xpath('//li[@class="global-views-breadcrumb-item"]/a[@data-touchpoint="home"]/text()')[0].strip() == 'Home' and html.xpath('//li[@class="global-views-breadcrumb-item-active"]/text()')[0].strip() == title:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                if html.xpath('//h3/text()')[0].strip() == 'Can\'t find the item you are looking for on this list? Download this':
                    raise

                # = = = = = = = = = = = = = = =

                dict_src, list_src = {}, html.xpath('//ul[@class="bxslider"]/li[@class="product-details-image-gallery-container"]/noscript/img[@itemprop="image"]/@src') + html.xpath('//div[@class="product-details-image-gallery"]/div[@class="product-details-image-gallery-detailed-image"]/noscript/img[@class="center-block"]/@src')
                for i, src in enumerate(list_src):
                    dict_src[str(i)] = src.strip()

                # = = = = = = = = = = = = = = =

                sku = html.xpath('//span[@itemprop="sku"]/text()')[0].strip()

                # = = = = = = = = = = = = = = =

                list_tr, dict_count = html.xpath('//form[@id="product-details-full-form"]/table[@class="specs-table table table-striped"]/tbody/tr'), {}
                for tr in list_tr:
                    key = tr.xpath('./td[1]/text()')[0].strip()
                    if key in dict_count:
                        dict_count[key] += 1
                    else:
                        dict_count[key] = 1
                
                dict_description, dict_no = {}, {}
                for i, tr in enumerate(list_tr):
                    key = tr.xpath('./td[1]/text()')[0].strip()
                    if dict_count[key] == 1:
                        dict_description[str(i)] = {key: tr.xpath('./td[2]/text()')[0].strip()}
                    else:
                        if key in dict_no:
                            dict_no[key] += 1
                        else:
                            dict_no[key] = 1
                        dict_description[str(i)] = {f'''{key}-{dict_no[key]}''': tr.xpath('./td[2]/text()')[0].strip()}

                # = = = = = = = = = = = = = = =

                list_oem = [text.strip() for text in html.xpath('//form[@id="product-details-full-form"]/div[@data-view="Additional.Detail.Fields"]/div/table[@class="specs-table table table-striped"]/tbody/tr/td[1]/text()')]
                list_oem = sorted(list(set(list_oem)), key=list_oem.index)
                oem = ';'.join(list_oem)

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'SKU': sku,
                                         'Title': title,
                                         'OEM': oem,
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': json.dumps(dict_src),
                                         'Json_Description': json.dumps(dict_description)}])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{request_url}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/3.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/3.part/{file}',
              f'''./file/3.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：12

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 4.https://www.auveco.com/metric-fast-asst-in-40-comp-bin-1040
[剩余数量：7] - [当前时间：09:30:17]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 3.https://www.auveco.com/metric-fastener-asst-reg-pitch-w-o-bin-1041
[剩余数量：6] - [当前时间：09:30:19]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 2.https://www.auveco.com/metric-fastener-assortment-in-two-40-compartment-bins-1042
[剩余数量：5] - [当前时间：09:30:19]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1.https://www.auveco.com/metric-fastener-assortment-w-o-bin-1043
[剩余数量：4] - [当前时间：09:30:19]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 6.https://www.auveco.com/gr-8-fastener-asst-in-40-compartment-bin-1030
[剩余数量：3] - [当前时间：09:30:20]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 9.https://www.auveco.com/import-retainer-assortment-w-o-bin-1055
[剩余数量：2] - [当前时间：09:30:20]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 5.https://www.auveco.com/uss-fastener-asst-wo-bin-1031
[剩余数量：1] - [当前时间：09:30:21]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 7.https://www.auveco.

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3953.16it/s]

Done ~
